In [1]:
import pandas as pd
import numpy as np
import re
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

nltk.download('words')
words = set(nltk.corpus.words.words())




[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/wcrossb/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package words to /Users/wcrossb/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [37]:
df = pd.read_csv("stored_vars.csv", lineterminator='\n')

def cleaner(tweet):
    tweet = re.sub("@[A-Za-z0-9]+","",str(tweet)) #Remove @ sign
    tweet = re.sub(r"(?:\@|http?\://|https?\://|www)\S+", "", str(tweet)) #Remove http links
    tweet = " ".join(tweet.split())
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet)
         if w.lower() in words or not w.isalpha())
    return tweet

df['tweet_clean'] = df['tweet'].apply(cleaner)


In [38]:
list1 = []
for i in df['tweet_clean']:
    list1.append((sid.polarity_scores(str(i)))['compound'])

df['sentiment'] = pd.Series(list1)

In [39]:
df_kws = pd.read_csv("kws.csv")
kw_list = df_kws.values.tolist()
print(kw_list[0])

df['keyword'] = np.nan
df['keyword'] = np.where(df['tweet_clean'].str.contains('Better financially') == True, 'Better financially', df['keyword'])
df.to_csv("temp2.csv")

print(df.dtypes)
for term in kw_list:
    df['keyword'] = np.where(df['tweet_clean'].str.contains(term[0], case=False) == True, term, df['keyword'])

df.to_csv("temp3.csv")


['Better financially']
Unnamed: 0       int64
date            object
tweet           object
tweet_clean     object
sentiment      float64
keyword         object
dtype: object


In [40]:
df_p = df.pivot_table(index='date', columns='keyword', values='sentiment', aggfunc='mean')
df_p.to_csv("temp4.csv")


In [45]:
df = df_p.reset_index()
df_c = df[df['date'].str.contains( "'" )==False ]

df_c['date'] = pd.to_datetime(df_c['date'], format='%Y-%m-%d %H:%M:%S')
df_c = df_c.set_index('date')
agg = df_c.resample('M').mean()

agg.to_csv("temp.csv")

agg = agg.reset_index()
save_date = agg[['date']].copy()

agg = agg.drop(columns = ['date','nan'])

In [59]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

agg_full = agg.apply(lambda row: row.fillna(row.mean()), axis=1)

#prepare the input series (x) by detrending and standardizing
x = StandardScaler().fit_transform(agg_full)

#Calculate the first  principal component & output explained variance
pca = PCA(n_components=1)
df_PC = pca.fit_transform(x)
print(pca.explained_variance_ratio_)


df_PC_merge = pd.DataFrame(df_PC, columns = ['Sentiment'])

df1 = pd.concat([save_date, df_PC_merge], axis=1)
df_final = df1.set_index('date')
df_final.to_csv("temp.csv")



[0.13955134]


In [62]:
import plotly.express as px
df_final.to_csv("export.csv")
df_final['Sentiment'] = df_final['Sentiment']*(-1)
fig = px.line(df_final, x=df_final.index, y=["Sentiment"], title='PC Sentiment')
fig.show()